In [2]:
! rm -rf data/lego-minifigures-classification
! pip install kaggle
! kaggle datasets download -d ihelon/lego-minifigures-classification -p data/
! unzip data/lego-minifigures-classification.zip -d data/lego-minifigures-classification

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
Archive:  data/lego-minifigures-classification.zip
  inflating: data/lego-minifigures-classification/LICENSE  
  inflating: data/lego-minifigures-classification/harry-potter/0001/001.jpg  
  inflating: data/lego-minifigures-classification/harry-potter/0001/002.jpg  
  inflating: data/lego-minifigures-classification/harry-potter/0001/003.jpg  
  inflating: data/lego-minifigures-classification/harry-potter/0001/004.jpg  
  inflating: data/lego-minifigures-classification/harry-potter/0001/005.jpg  
  inflatin

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
import shutil
import os
from PIL import Image

In [4]:
train_set = pd.read_csv("data/lego-minifigures-classification/index.csv",sep=",", encoding="ISO-8859-1", low_memory=False,)
metadata = pd.read_csv("data/lego-minifigures-classification/metadata.csv",sep=",", encoding="ISO-8859-1", low_memory=False,)

In [5]:
root_path = "data/lego-minifigures-classification/"

In [6]:
for i in list(train_set.itertuples()):
    path = i.path
    classe = i.class_id
    train_valid = i._4
    classe_name = metadata[metadata.class_id == classe].minifigure_name.values[0]
    if os.path.isdir(str(root_path)+str(train_valid)+"/"+str(classe_name)) is False:
        os.makedirs(str(root_path)+str(train_valid)+"/"+str(classe_name))
    shutil.move(str(root_path)+str(path), str(root_path)+str(train_valid)+"/"+str(classe_name)+"/"+str(path.split('/')[2]))

In [7]:
batch_size = 10
train_batches = ImageDataGenerator(rescale=1./255) \
    .flow_from_directory(directory=str(root_path)+"train", target_size=(224,224), batch_size=batch_size)
valid_batches = ImageDataGenerator(rescale=1./255) \
    .flow_from_directory(directory=str(root_path)+"valid", target_size=(224,224), batch_size=batch_size)
input_shape = train_batches.image_shape
num_classes = train_batches.num_classes

Found 168 images belonging to 28 classes.
Found 157 images belonging to 28 classes.


In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
      if(logs.get('loss')<=0.01):
          self.model.stop_training = True

model = Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), input_shape=input_shape))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(32, (3, 3)))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3)))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dense(128))
model.add(keras.layers.Activation('relu'))
# model.add(keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(num_classes, activation='sigmoid'))
model.compile(optimizer=Adam(), 
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
model.fit(x=train_batches, epochs=200,callbacks=[myCallback()], validation_data=valid_batches)

Epoch 1/200
15/15 [==============================] - 11s 758ms/step - loss: 0.3427 - accuracy: 0.0470 - val_loss: 0.3836 - val_accuracy: 0.0362
Epoch 2/200
15/15 [==============================] - 11s 738ms/step - loss: 0.1505 - accuracy: 0.2349 - val_loss: 0.2441 - val_accuracy: 0.0435
Epoch 3/200
15/15 [==============================] - 11s 734ms/step - loss: 0.1062 - accuracy: 0.5638 - val_loss: 0.2386 - val_accuracy: 0.0870
Epoch 4/200
15/15 [==============================] - 11s 733ms/step - loss: 0.0619 - accuracy: 0.7919 - val_loss: 0.1898 - val_accuracy: 0.1087
Epoch 5/200
15/15 [==============================] - 11s 737ms/step - loss: 0.0319 - accuracy: 0.9128 - val_loss: 0.1814 - val_accuracy: 0.0870
Epoch 6/200
15/15 [==============================] - 11s 738ms/step - loss: 0.0216 - accuracy: 0.9463 - val_loss: 0.1752 - val_accuracy: 0.0797
Epoch 7/200
15/15 [==============================] - 11s 738ms/step - loss: 0.0197 - accuracy: 0.9597 - val_loss: 0.1782 - val_accuracy:

In [16]:
train_batches = ImageDataGenerator(preprocessing_function=keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=str(root_path)+"train", target_size=(224,224), batch_size=batch_size)
valid_batches = ImageDataGenerator(preprocessing_function=keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=str(root_path)+"valid", target_size=(224,224), batch_size=batch_size)
input_shape = train_batches.image_shape
num_classes = train_batches.num_classes

Found 168 images belonging to 28 classes.
Found 157 images belonging to 28 classes.


In [19]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
      if(logs.get('loss')<=0.01):
          self.model.stop_training = True

model2 = Sequential()
vgg16_model = keras.applications.vgg16.VGG16()
for layer in vgg16_model.layers[:-1]:
  model2.add(layer)
for layer in model2.layers:
  layer.trainable = False
model2.add(keras.layers.Dense(num_classes, activation="sigmoid"))
model2.compile(optimizer=Adam(), 
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
model2.fit(x=train_batches, epochs=200,callbacks=[myCallback()], validation_data=valid_batches)

Epoch 1/200
17/17 [==============================] - 161s 9s/step - loss: 0.3104 - accuracy: 0.1131 - val_loss: 0.2304 - val_accuracy: 0.2803
Epoch 2/200
17/17 [==============================] - 164s 10s/step - loss: 0.1108 - accuracy: 0.5655 - val_loss: 0.1038 - val_accuracy: 0.5032
Epoch 3/200
17/17 [==============================] - 161s 9s/step - loss: 0.0354 - accuracy: 0.9345 - val_loss: 0.0990 - val_accuracy: 0.5796
Epoch 4/200
17/17 [==============================] - 162s 10s/step - loss: 0.0164 - accuracy: 0.9881 - val_loss: 0.0934 - val_accuracy: 0.6561
Epoch 5/200
17/17 [==============================] - 161s 9s/step - loss: 0.0081 - accuracy: 1.0000 - val_loss: 0.0898 - val_accuracy: 0.6815
